# Workshop Azure Databricks
## 06. Classification
<img src="https://raw.githubusercontent.com/retkowsky/images/master/AzureDatabricksLogo.jpg"><br>

# Documentation
Présentation https://azure.microsoft.com/fr-fr/services/databricks/

Documentation Azure Databricks : https://docs.microsoft.com/fr-fr/azure/databricks/

Documentation Azure ML : https://docs.microsoft.com/en-us/azure/machine-learning/

Github : https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/azure-databricks

## Creating a Classification Model

In this exercise, you will implement a classification model that uses features of a flight to predict whether or not it will be late.

### Import Spark SQL and Spark ML Libraries

First, import the libraries you will need:

In [0]:
import datetime
now = datetime.datetime.now()
print(now)

2021-02-10 13:55:07.059675

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer, VectorAssembler

### Load Source Data
The data for this exercise is provided as a CSV file containing details of flights that has already been cleaned up for modeling. The data includes specific characteristics (or *features*) for each flight, as well as a *label* column indicating whether or not the flight was late (a flight with an arrival delay of more than 25 minutes is considered *late*).

You will load this data into a dataframe and display it.

> Importer le fichier CSV dans DBFS

In [0]:
# File location and type
file_location = "/FileStore/tables/flights.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
data = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(data)

DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,DepDelay,ArrDelay,Late
21,2,WN,10721,13342,26,57,1
13,1,AA,15016,12892,51,27,1
5,5,FL,10397,11433,9,4,0
22,1,US,11278,14100,35,71,1
23,4,WN,12451,10693,9,5,0
5,7,AA,11298,15016,39,42,1
4,6,UA,13930,14307,71,58,1
10,3,9E,14307,11433,68,140,1
29,7,UA,14057,14771,130,125,1
14,7,UA,14771,11292,20,42,1


### Split the Data
It is common practice when building supervised machine learning models to split the source data, using some of it to train the model and reserving some to test the trained model. In this exercise, you will use 70% of the data for training, and reserve 30% for testing.

In [0]:
splits = data.randomSplit([0.7, 0.3])

train = splits[0]
test = splits[1]

train_rows = train.count()
test_rows = test.count()

print ("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 445743 Testing Rows: 191340

### Prepare the Training Data
To train the classification model, you need a training data set that includes a vector of numeric features, and a label column. In this exercise, you will use the **StringIndexer** class to generate a numeric category for each discrete **Carrier** string value, and then use the **VectorAssembler** class to transform the numeric features that would be available for a flight that hasn't yet arrived into a vector, and then rename the **Late** column to **label** as this is what we're going to try to predict.

*Note: This is a deliberately simple example. In reality you'd likely perform mulitple data preparation steps, and later in this course we'll examine how to encapsulate these steps in to a pipeline. For now, we'll just use the numeric features as they are to define the training dataset.*

In [0]:
# Carrier is a string, and we need our features to be numeric - so we'll generate a numeric index for each distinct carrier string, and transform the dataframe to add that as a column

carrierIndexer = StringIndexer(inputCol="Carrier", outputCol="CarrierIdx")
numTrain = carrierIndexer.fit(train).transform(train)

# Now we'll assemble a vector of all the numeric feature columns (other than ArrDelay, which we wouldn't have for enroute flights)
assembler = VectorAssembler(inputCols = ["DayofMonth", "DayOfWeek", "CarrierIdx", "OriginAirportID", "DestAirportID", "DepDelay"], outputCol="features")

training = assembler.transform(numTrain).select(col("features"), col("Late").alias("label"))

training.show()

+--------------------+-----+
 features|label|
+--------------------+-----+
[1.0,1.0,10.0,103...| 1|
[1.0,1.0,10.0,103...| 0|
[1.0,1.0,10.0,107...| 1|
[1.0,1.0,10.0,110...| 0|
[1.0,1.0,10.0,110...| 1|
[1.0,1.0,10.0,110...| 0|
[1.0,1.0,10.0,111...| 1|
[1.0,1.0,10.0,111...| 1|
[1.0,1.0,10.0,111...| 1|
[1.0,1.0,10.0,111...| 1|
[1.0,1.0,10.0,111...| 0|
[1.0,1.0,10.0,111...| 0|
[1.0,1.0,10.0,111...| 0|
[1.0,1.0,10.0,111...| 1|
[1.0,1.0,10.0,111...| 0|
[1.0,1.0,10.0,111...| 1|
[1.0,1.0,10.0,112...| 1|
[1.0,1.0,10.0,114...| 0|
[1.0,1.0,10.0,114...| 1|
[1.0,1.0,10.0,114...| 1|
+--------------------+-----+
only showing top 20 rows

### Train a Classification Model
Next, you need to train a classification model using the training data. To do this, create an instance of the classification algorithm you want to use and use its **fit** method to train a model based on the training dataframe. In this exercise, you will use a *Logistic Regression* classification algorithm - but you can use the same technique for any of the classification algorithms supported in the spark.ml API.

In [0]:
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)
model = lr.fit(training)

print ("OK !")

OK !

### Prepare the Testing Data
Now that you have a trained model, you can test it using the testing data you reserved previously. First, you need to prepare the testing data in the same way as you did the training data by transforming the feature columns into a vector. This time you'll rename the **Late** column to **trueLabel**.

In [0]:
# Transform the test data to add the numeric carrier index
numTest = carrierIndexer.fit(test).transform(test)

# Generate the features vector and label
testing = assembler.transform(numTest).select(col("features"), col("Late").alias("trueLabel"))
testing.show()

+--------------------+---------+
 features|trueLabel|
+--------------------+---------+
[1.0,1.0,10.0,105...| 1|
[1.0,1.0,10.0,107...| 0|
[1.0,1.0,10.0,107...| 1|
[1.0,1.0,10.0,111...| 1|
[1.0,1.0,10.0,124...| 0|
[1.0,1.0,10.0,124...| 1|
[1.0,1.0,10.0,124...| 0|
[1.0,1.0,10.0,124...| 1|
[1.0,1.0,10.0,124...| 1|
[1.0,1.0,10.0,132...| 0|
[1.0,1.0,10.0,132...| 1|
[1.0,1.0,10.0,134...| 0|
[1.0,1.0,10.0,134...| 1|
[1.0,1.0,10.0,139...| 0|
[1.0,1.0,2.0,1072...| 0|
[1.0,1.0,2.0,1127...| 1|
[1.0,1.0,2.0,1129...| 0|
[1.0,1.0,2.0,1129...| 1|
[1.0,1.0,2.0,1129...| 0|
[1.0,1.0,2.0,1129...| 0|
+--------------------+---------+
only showing top 20 rows

### Test the Model
Now you're ready to use the **transform** method of the model to generate some predictions. You can use this approach to predict delay status for flights where the label is unknown; but in this case you are using the test data which includes a known true label value, so you can compare the predicted status to the actual status.

In [0]:
prediction = model.transform(testing)
predicted = prediction.select("features", "probability", col("prediction").cast("Int"), "trueLabel")
predicted.show(100, truncate=False)

+------------------------------------+----------------------------------------+----------+---------+
features |probability |prediction|trueLabel|
+------------------------------------+----------------------------------------+----------+---------+
[1.0,1.0,10.0,10529.0,14492.0,20.0] |[0.488117202739121,0.511882797260879] |1 |1 |
[1.0,1.0,10.0,10721.0,11193.0,-5.0] |[0.5922385558122495,0.4077614441877505] |0 |0 |
[1.0,1.0,10.0,10721.0,12478.0,25.0] |[0.4594734506512657,0.5405265493487343] |1 |1 |
[1.0,1.0,10.0,11193.0,11298.0,55.0] |[0.32690873007749877,0.6730912699225012]|1 |1 |
[1.0,1.0,10.0,12478.0,10721.0,-5.0] |[0.5978101934486494,0.40218980655135067]|0 |0 |
[1.0,1.0,10.0,12478.0,11278.0,89.0] |[0.20964896198759853,0.7903510380124015]|1 |1 |
[1.0,1.0,10.0,12478.0,11433.0,-1.0] |[0.5822266398162579,0.41777336018374206]|0 |0 |
[1.0,1.0,10.0,12478.0,12264.0,129.0]|[0.11398083392378681,0.8860191660762131]|1 |1 |
[1.0,1.0,10.0,12478.0,13930.0,-1.0] |[0.590065152867076,0.409934847132924] |0 |1 |
[1.0,1.0,10.0,13232.0,11433.0,-4.0] |[0.5986147322533605,0.40138526774663963]|0 |0 |
[1.0,1.0,10.0,13244.0,11193.0,-4.0] |[0.5979162363181839,0.40208376368181603]|0 |1 |
[1.0,1.0,10.0,13487.0,11193.0,-7.0] |[0.6120802234779023,0.3879197765220977] |0 |0 |
[1.0,1.0,10.0,13487.0,14683.0,65.0] |[0.30484388943302837,0.6951561105669716]|1 |1 |
[1.0,1.0,10.0,13931.0,10721.0,-11.0]|[0.6297129830229505,0.3702870169770495] |0 |0 |
[1.0,1.0,2.0,10721.0,13303.0,-8.0] |[0.615832101680511,0.38416789831948905] |0 |0 |
[1.0,1.0,2.0,11278.0,13303.0,0.0] |[0.5827021774424695,0.41729782255753045]|0 |1 |
[1.0,1.0,2.0,11298.0,10140.0,6.0] |[0.5455761698392532,0.4544238301607468] |0 |0 |
[1.0,1.0,2.0,11298.0,10423.0,78.0] |[0.24248719425243906,0.757512805747561] |1 |1 |
[1.0,1.0,2.0,11298.0,10693.0,0.0] |[0.5745482176046781,0.42545178239532194]|0 |0 |
[1.0,1.0,2.0,11298.0,10821.0,-1.0] |[0.5794459808100472,0.4205540191899529] |0 |0 |
[1.0,1.0,2.0,11298.0,11057.0,48.0] |[0.3592622341856197,0.6407377658143802] |1 |1 |
[1.0,1.0,2.0,11298.0,11618.0,82.0] |[0.2319613244970289,0.7680386755029711] |1 |1 |
[1.0,1.0,2.0,11298.0,12451.0,24.0] |[0.47036666606001,0.52963333393999] |1 |0 |
[1.0,1.0,2.0,11298.0,12892.0,75.0] |[0.2588588891378401,0.74114111086216] |1 |1 |
[1.0,1.0,2.0,11298.0,13204.0,34.0] |[0.4272612255344691,0.5727387744655309] |1 |1 |
[1.0,1.0,2.0,11298.0,13495.0,-1.0] |[0.5878547618574945,0.41214523814250553]|0 |0 |
[1.0,1.0,2.0,11298.0,13891.0,65.0] |[0.298411556931303,0.701588443068697] |1 |1 |
[1.0,1.0,2.0,11298.0,14057.0,3.0] |[0.5716862820754718,0.42831371792452827]|0 |0 |
[1.0,1.0,2.0,11298.0,14492.0,69.0] |[0.28480748371669456,0.7151925162833054]|1 |1 |
[1.0,1.0,2.0,11298.0,14679.0,0.0] |[0.5871063133963742,0.4128936866036258] |0 |0 |
[1.0,1.0,2.0,11298.0,14683.0,57.0] |[0.33240249633419894,0.6675975036658011]|1 |1 |
[1.0,1.0,2.0,11618.0,11298.0,-10.0] |[0.6219167496100649,0.378083250389935] |0 |0 |
[1.0,1.0,2.0,12451.0,11298.0,-5.0] |[0.6033628240352962,0.3966371759647038] |0 |0 |
[1.0,1.0,2.0,12889.0,11298.0,21.0] |[0.4870296359359416,0.5129703640640584] |1 |0 |
[1.0,1.0,2.0,12892.0,11298.0,50.0] |[0.35761020011309547,0.6423897998869045]|1 |1 |
[1.0,1.0,2.0,12892.0,12478.0,0.0] |[0.5866298904294555,0.41337010957054454]|0 |1 |
[1.0,1.0,2.0,12953.0,11298.0,63.0] |[0.3048992928916629,0.695100707108337] |1 |1 |
[1.0,1.0,2.0,13303.0,11298.0,17.0] |[0.5071587436088614,0.4928412563911386] |0 |1 |
[1.0,1.0,2.0,13303.0,13204.0,47.0] |[0.3777718394838249,0.6222281605161751] |1 |1 |
[1.0,1.0,2.0,13303.0,15304.0,36.0] |[0.4330733019122141,0.5669266980877858] |1 |1 |
[1.0,1.0,2.0,13303.0,15304.0,55.0] |[0.34998044800416467,0.6500195519958354]|1 |1 |
[1.0,1.0,2.0,13930.0,11298.0,-4.0] |[0.6048443413229314,0.39515565867706864]|0 |0 |
[1.0,1.0,2.0,13930.0,12953.0,-7.0] |[0.6230092879902347,0.3769907120097653] |0 |1 |
[1.0,1.0,2.0,13930.0,12953.0,5.0] |[0.5698926569962786,0.43010734300372133]|0 |1 |
[1.0,1.0,2.0,13930.0,12953.0,96.0] |[0.19877712505365724,0.8012

In [0]:
display(predicted)

features,probability,prediction,trueLabel
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 10529.0, 14492.0, 20.0))","Map(vectorType -> dense, length -> 2, values -> List(0.488117202739121, 0.511882797260879))",1,1
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 10721.0, 11193.0, -5.0))","Map(vectorType -> dense, length -> 2, values -> List(0.5922385558122495, 0.4077614441877505))",0,0
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 10721.0, 12478.0, 25.0))","Map(vectorType -> dense, length -> 2, values -> List(0.4594734506512657, 0.5405265493487343))",1,1
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 11193.0, 11298.0, 55.0))","Map(vectorType -> dense, length -> 2, values -> List(0.32690873007749877, 0.6730912699225012))",1,1
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 12478.0, 10721.0, -5.0))","Map(vectorType -> dense, length -> 2, values -> List(0.5978101934486494, 0.40218980655135067))",0,0
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 12478.0, 11278.0, 89.0))","Map(vectorType -> dense, length -> 2, values -> List(0.20964896198759853, 0.7903510380124015))",1,1
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 12478.0, 11433.0, -1.0))","Map(vectorType -> dense, length -> 2, values -> List(0.5822266398162579, 0.41777336018374206))",0,0
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 12478.0, 12264.0, 129.0))","Map(vectorType -> dense, length -> 2, values -> List(0.11398083392378681, 0.8860191660762131))",1,1
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 12478.0, 13930.0, -1.0))","Map(vectorType -> dense, length -> 2, values -> List(0.590065152867076, 0.409934847132924))",0,1
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 13232.0, 11433.0, -4.0))","Map(vectorType -> dense, length -> 2, values -> List(0.5986147322533605, 0.40138526774663963))",0,0


In [0]:
display(predicted)

features,probability,prediction,trueLabel
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 10529.0, 14492.0, 20.0))","Map(vectorType -> dense, length -> 2, values -> List(0.488117202739121, 0.511882797260879))",1,1
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 10721.0, 11193.0, -5.0))","Map(vectorType -> dense, length -> 2, values -> List(0.5922385558122495, 0.4077614441877505))",0,0
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 10721.0, 12478.0, 25.0))","Map(vectorType -> dense, length -> 2, values -> List(0.4594734506512657, 0.5405265493487343))",1,1
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 11193.0, 11298.0, 55.0))","Map(vectorType -> dense, length -> 2, values -> List(0.32690873007749877, 0.6730912699225012))",1,1
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 12478.0, 10721.0, -5.0))","Map(vectorType -> dense, length -> 2, values -> List(0.5978101934486494, 0.40218980655135067))",0,0
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 12478.0, 11278.0, 89.0))","Map(vectorType -> dense, length -> 2, values -> List(0.20964896198759853, 0.7903510380124015))",1,1
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 12478.0, 11433.0, -1.0))","Map(vectorType -> dense, length -> 2, values -> List(0.5822266398162579, 0.41777336018374206))",0,0
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 12478.0, 12264.0, 129.0))","Map(vectorType -> dense, length -> 2, values -> List(0.11398083392378681, 0.8860191660762131))",1,1
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 12478.0, 13930.0, -1.0))","Map(vectorType -> dense, length -> 2, values -> List(0.590065152867076, 0.409934847132924))",0,1
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 13232.0, 11433.0, -4.0))","Map(vectorType -> dense, length -> 2, values -> List(0.5986147322533605, 0.40138526774663963))",0,0


In [0]:
display(predicted)

features,probability,prediction,trueLabel
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 10529.0, 14492.0, 20.0))","Map(vectorType -> dense, length -> 2, values -> List(0.488117202739121, 0.511882797260879))",1,1
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 10721.0, 11193.0, -5.0))","Map(vectorType -> dense, length -> 2, values -> List(0.5922385558122495, 0.4077614441877505))",0,0
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 10721.0, 12478.0, 25.0))","Map(vectorType -> dense, length -> 2, values -> List(0.4594734506512657, 0.5405265493487343))",1,1
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 11193.0, 11298.0, 55.0))","Map(vectorType -> dense, length -> 2, values -> List(0.32690873007749877, 0.6730912699225012))",1,1
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 12478.0, 10721.0, -5.0))","Map(vectorType -> dense, length -> 2, values -> List(0.5978101934486494, 0.40218980655135067))",0,0
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 12478.0, 11278.0, 89.0))","Map(vectorType -> dense, length -> 2, values -> List(0.20964896198759853, 0.7903510380124015))",1,1
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 12478.0, 11433.0, -1.0))","Map(vectorType -> dense, length -> 2, values -> List(0.5822266398162579, 0.41777336018374206))",0,0
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 12478.0, 12264.0, 129.0))","Map(vectorType -> dense, length -> 2, values -> List(0.11398083392378681, 0.8860191660762131))",1,1
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 12478.0, 13930.0, -1.0))","Map(vectorType -> dense, length -> 2, values -> List(0.590065152867076, 0.409934847132924))",0,1
"Map(vectorType -> dense, length -> 6, values -> List(1.0, 1.0, 10.0, 13232.0, 11433.0, -4.0))","Map(vectorType -> dense, length -> 2, values -> List(0.5986147322533605, 0.40138526774663963))",0,0


> You can open Lab07